In [71]:
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr

import warnings
warnings.filterwarnings("ignore")

read_path = os.getcwd() + '/Output/'
write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_region = 'Region'
col_year = 'Year'
col_gdp = 'GDP'
col_shift = 'Shift'

In [72]:
df = pd.read_csv(read_path + 'GoldDataframe.csv', index_col = [col_country, col_region, col_year])
df

AgriShareGDP  CreditToAgriFishForest  \
Country     Region             Year                                         
Afghanistan South Asia         2000      54.06300                     NaN   
                               2001      54.06300                     NaN   
                               2002      45.13440                     NaN   
                               2003      41.90340                     NaN   
                               2004      35.61280                     NaN   
...                                           ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016       7.87399                     NaN   
                               2017       8.34095                     NaN   
                               2018       8.30469                     NaN   
                               2019       8.17322                     NaN   
                               2020      10.93630                     NaN   

                                     EmploymentRural           GDP  \
Country     Region             Year                                  
Afghanistan South Asia         2000              NaN   3342.034168   
                               2001              NaN   3598.470576   
                               2002              NaN   4141.523943   
                               2003              NaN   4729.042179   
                               2004              NaN   5388.482107   
...                                              ...           ...   
Zimbabwe    Sub-Saharan Africa 2016              NaN  20548.678073   
                               2017              NaN  22040.902301   
                               2018              NaN  24311.560545   
                               2019              NaN  21935.075306   
                               2020              NaN  21786.743637   

                                     % Soldiers  Employment in industry  \
Country     Region             Year                                       
Afghanistan South Asia         2000    7.887961                    9.48   
                               2001    5.020511                    8.98   
                               2002    2.153062                    9.99   
                               2003    2.208290                   10.35   
                               2004    0.435599                   10.61   
...                                         ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016    0.759750                    7.05   
                               2017    0.750720                    6.90   
                               2018    0.738210                    6.75   
                               2019    0.738210                    6.57   
                               2020    0.738210                    6.57   

                                     Employment in services  Birth Rate  \
Country     Region             Year                                       
Afghanistan South Asia         2000               24.680000      48.021   
                               2001               24.719999      47.505   
                               2002               25.590000      46.901   
                               2003               25.950001      46.231   
                               2004               26.120001      45.507   
...                                                     ...         ...   
Zimbabwe    Sub-Saharan Africa 2016               26.070000      32.864   
                               2017               26.629999      31.732   
                               2018               27.230000      30.676   
                               2019               27.240000      29.747   
                               2020               27.240000      28.977   

                                     Cost business start-up  Death Rate  ...  \
Country     Region             Year                                      ...   
Afghanistan 

In [98]:
country_list = list(set(df.index.get_level_values(col_country)))
region_list = list(set(df.index.get_level_values(col_region)))

country_list.sort()
region_list.sort()

total_years = len(set(df.index.get_level_values(col_year)))

#shifts = range(-total_years + 1, total_years)
shifts = range(0, total_years)

In [92]:
def norm (df: pd.DataFrame):    
    norm_df = pd.DataFrame(index = df.index)

    for country in country_list:
        for col_name in df.columns:
            # Select indicator col_name for country country
            col = df.loc[df.index.get_level_values(col_country) == country, col_name]
            # Normalize values
            col = col.map(lambda x: (x - col.min()) / (col.max() - col.min()))
            # Add them to the normalized DataFrame
            norm_df.loc[df.index.get_level_values(col_country) == country, col_name] = col

    return norm_df

In [87]:
norm_df = norm(df)

In [103]:
min_year = df.index.get_level_values(col_year).min()
max_year = df.index.get_level_values(col_year).max()

corr_df_list = []

for shift in shifts:
    corr_s_list = []
    for region in region_list:
        min_year_gdp = min_year + max(shift, 0)
        max_year_gdp = max_year - min(shift, 0)

        min_year_ind = min_year + min(shift, 0)
        max_year_ind = max_year - max(shift, 0)

        #print(f"MIN GDP {min_year_gdp} - MAX GDP {max_year_gdp}")
        #print(f"MIN IND {min_year_ind} - MAX IND {max_year_ind}")
        gdp_s = norm_df.loc[(norm_df.index.get_level_values(col_year) >= min_year_gdp) & (norm_df.index.get_level_values(col_region) == region), col_gdp]
        ind_df = norm_df.loc[(norm_df.index.get_level_values(col_year) <= max_year_ind) & (norm_df.index.get_level_values(col_region) == region)].drop(columns = col_gdp)

        gdp_s.index = gdp_s.index.map(lambda x: (x[0], x[1], x[2] - shift)) #NOT ROBUST ENOUGH

        corr_s = ind_df.apply(lambda x: x.corr(gdp_s, method = 'spearman'), axis = 0)
        corr_s.name = (region, shift)
        corr_s_list.append(corr_s)
        
    aux_corr_df = pd.concat(corr_s_list, axis = 1)
    corr_df_list.append(aux_corr_df)
    
corr_df = pd.concat(corr_df_list, axis = 1).transpose().sort_index()
corr_df.index.names = [col_region, col_shift]
display(corr_df)

corr_df.to_csv(write_path + 'Shift_Corr_Dataframe.csv')

AgriShareGDP  CreditToAgriFishForest  \
Region                Shift                                         
East Asia and Pacific 0         -0.557078                0.035495   
                      1         -0.571000                0.045581   
                      2         -0.564104                0.061005   
                      3         -0.533058                0.063054   
                      4         -0.490044                0.107678   
...                                   ...                     ...   
Sub-Saharan Africa    16         0.005624                0.019947   
                      17        -0.003210               -0.017634   
                      18        -0.015317               -0.013647   
                      19        -0.016207               -0.081794   
                      20        -0.052106                0.009031   

                             EmploymentRural  % Soldiers  \
Region                Shift                                
East Asia and Pacific 0             0.505195   -0.482020   
                      1             0.440602   -0.478531   
                      2             0.492982   -0.469527   
                      3             0.550052   -0.452155   
                      4             0.529412   -0.431361   
...                                      ...         ...   
Sub-Saharan Africa    16                 NaN   -0.028297   
                      17                 NaN    0.076304   
                      18                 NaN    0.021263   
                      19                 NaN    0.104081   
                      20                 NaN    0.185899   

                             Employment in industry  Employment in services  \
Region                Shift                                                   
East Asia and Pacific 0                    0.257993                0.777541   
                      1                    0.263635                0.768111   
                      2                    0.275608                0.752348   
                      3                    0.279594                0.730858   
                      4                    0.272671                0.705514   
...                                             ...                     ...   
Sub-Saharan Africa    16                  -0.142260                0.141987   
                      17                  -0.161870                0.066625   
                      18                  -0.168480               -0.085154   
                      19                  -0.130423               -0.165970   
                      20                  -0.058761               -0.200469   

                             Birth Rate  Cost business start-up  Death Rate  \
Region                Shift                                                   
East Asia and Pacific 0       -0.648727               -0.803675   -0.054715   
                      1       -0.610870               -0.786266   -0.071897   
                      2       -0.566774               -0.764718   -0.081032   
                      3       -0.521940               -0.752843   -0.087992   
                      4       -0.476209               -0.738602   -0.085950   
...                                 ...                     ...         ...   
Sub-Saharan Africa    16      -0.299483               -0.086779   -0.338583   
                      17      -0.158429                0.016016   -0.266506   
                      18      -0.001722                0.027980   -0.126855   
                      19       0.156525                0.054790    0.092861   
                      20      -0.068548               -0.066102    0.476588   

                             Employment in agriculture  ...  \
Region                Shift                             ...   
East Asia and Pacific 0                      -0.873590  ...   
                      1                      -0.860879  ...   
                      2                   

In [95]:
min_year = df.index.get_level_values(col_year).min()
max_year = df.index.get_level_values(col_year).max()

corr_df_list = []

for shift in shifts:
    corr_s_list = []
    for region in region_list:
        min_year_gdp = min_year + shift
        max_year_gdp = max_year

        min_year_ind = min_year
        max_year_ind = max_year - shift

        gdp_s = norm_df.loc[(norm_df.index.get_level_values(col_year) >= min_year_gdp) & (norm_df.index.get_level_values(col_region) == region), col_gdp]
        ind_df = norm_df.loc[(norm_df.index.get_level_values(col_year) <= max_year_ind) & (norm_df.index.get_level_values(col_region) == region)].drop(columns = col_gdp)

        gdp_s.index = gdp_s.index.map(lambda x: (x[0], x[1], x[2] - shift))

        corr_s = ind_df.apply(lambda x: x.corr(gdp_s, method = 'spearman'), axis = 0)
        corr_s.name = (region, shift)
        corr_s_list.append(corr_s)

    for region in region_list:
        min_year_gdp = min_year
        max_year_gdp = max_year - shift

        min_year_ind = min_year + shift
        max_year_ind = max_year

        gdp_s = norm_df.loc[(norm_df.index.get_level_values(col_year) >= min_year_gdp) & (norm_df.index.get_level_values(col_region) == region), col_gdp]
        ind_df = norm_df.loc[(norm_df.index.get_level_values(col_year) <= max_year_ind) & (norm_df.index.get_level_values(col_region) == region)].drop(columns = col_gdp)

        gdp_s.index = gdp_s.index.map(lambda x: (x[0], x[1], x[2] - shift))

        corr_s = ind_df.apply(lambda x: x.corr(gdp_s, method = 'spearman'), axis = 0)
        corr_s.name = (region, -shift)
        corr_s_list.append(corr_s)
        
    aux_corr_df = pd.concat(corr_s_list, axis = 1)
    corr_df_list.append(aux_corr_df)

corr_df = pd.concat(corr_df_list, axis = 1).transpose().sort_index()
corr_df.index.names = [col_region, col_shift]
display(corr_df)

corr_df.to_csv(write_path + 'Shift_Corr_Dataframe.csv')

AgriShareGDP  CreditToAgriFishForest  \
Region                Shift                                         
East Asia and Pacific -21             NaN                     NaN   
                      -20        0.139208               -0.357576   
                      -20       -0.259753               -0.340427   
                      -19       -0.156915               -0.136097   
                      -19       -0.220394                0.098397   
...                                   ...                     ...   
Sub-Saharan Africa     19       -0.081918               -0.068120   
                       19       -0.016207               -0.081794   
                       20       -0.211637               -0.078289   
                       20       -0.052106                0.009031   
                       21             NaN                     NaN   

                             EmploymentRural  % Soldiers  \
Region                Shift                                
East Asia and Pacific -21                NaN         NaN   
                      -20                NaN    0.240563   
                      -20                NaN    0.223921   
                      -19               -1.0    0.156956   
                      -19                1.0    0.375826   
...                                      ...         ...   
Sub-Saharan Africa     19                NaN   -0.066470   
                       19                NaN    0.104081   
                       20                NaN   -0.091135   
                       20                NaN    0.185899   
                       21                NaN         NaN   

                             Employment in industry  Employment in services  \
Region                Shift                                                   
East Asia and Pacific -21                       NaN                     NaN   
                      -20                 -0.205633               -0.071524   
                      -20                 -0.421776                0.169325   
                      -19                  0.008806                0.008945   
                      -19                 -0.487082               -0.358780   
...                                             ...                     ...   
Sub-Saharan Africa     19                 -0.114484                0.019038   
                       19                 -0.130423               -0.165970   
                       20                 -0.202122               -0.001635   
                       20                 -0.058761               -0.200469   
                       21                       NaN                     NaN   

                             Birth Rate  Cost business start-up  Death Rate  \
Region                Shift                                                   
East Asia and Pacific -21           NaN                     NaN         NaN   
                      -20     -0.300662               -0.100233    0.248047   
                      -20     -0.368011               -0.009174    0.186002   
                      -19      0.242989               -0.102161   -0.145354   
                      -19      0.188423                0.088243    0.330078   
...                                 ...                     ...         ...   
Sub-Saharan Africa     19     -0.171689                0.001453   -0.276631   
                       19      0.156525                0.054790    0.092861   
                       20      0.080734               -0.024553    0.166875   
                       20     -0.068548               -0.066102    0.476588   
                       21           NaN                     NaN         NaN   

                             Employment in agriculture  ...  \
Region                Shift                             ...   
East Asia and Pacific -21                          NaN  ...   
                      -20                    -0.242844  ...   
                      -20                 

In [ ]:
min_year = df.index.get_level_values(col_year).min()
max_year = df.index.get_level_values(col_year).max()

corr_s_list = []

for shift in shifts:
    shift = 16
    display(shift)
    for region in region_list:
        display(region)
        min_year_gdp = min_year + shift
        max_year_gdp = max_year

        min_year_ind = min_year
        max_year_ind = max_year - shift

        gdp_df = df.loc[(df.index.get_level_values(col_year) >= min_year_gdp) & (df.index.get_level_values(col_region) == region), [col_gdp]]
        ind_df = df.loc[(df.index.get_level_values(col_year) <= max_year_ind) & (df.index.get_level_values(col_region) == region)].drop(columns = col_gdp)

        norm_gdp_df = norm(gdp_df) #.reset_index().drop(columns = col_year)
        norm_ind_df = norm(ind_df) #.reset_index().drop(columns = col_year)

        norm_gdp_s = norm_gdp_df.loc[:, col_gdp].convert_dtypes(float)
        norm_gdp_s.index = norm_gdp_df.index.map(lambda x: (x[0], x[1], x[2] - shift))

        #norm_gdp_df.index = norm_gdp_df.index.map(lambda x: (x[0], x[1], x[2] - min_year_gdp))
        #norm_ind_df.index = norm_ind_df.index.map(lambda x: (x[0], x[1], x[2] - min_year_ind))

        display(norm_ind_df.apply(lambda x: pd.Series([1, 2, 3], index = ['a', 'b', 'c'])))

        display(norm_gdp_s)
        display(norm_ind_df)

        print('Display deez:')
        
        pop_s = norm_ind_df['Population'].convert_dtypes(float)
        display(norm_gdp_s)
        display(pop_s)
        print('Corr:')
        display(pop_s.corr(norm_gdp_s, method = 'spearman'))
        x_list = []
        print('Apply:')
        display(norm_ind_df.apply(lambda x: x_list.append(x), axis = 0))
        print('List:')
        display(x_list)
        for x in x_list:
            display(x.corr(norm_gdp_s, method = 'spearman'))
        corr_s = norm_ind_df.apply(lambda x: x.corr(norm_gdp_s, method = 'spearman'), axis = 0)
        
        display(corr_s)
        corr_s_list.append(corr_s)
        
        #corr_df = norm_ind_df.drop(columns = [col_country, col_region]).corrwith(norm_gdp_df[col_gdp], method = 'spearman')
        break

    break
corr_df = pd.DataFrame(corr_s_list)
#display(norm_gdp_df[col_gdp])
#display(norm_ind_df)

#display(corr_df)

In [26]:
norm_ind_df['Population']

pandas.core.series.Series

In [6]:
PVALUE_VAR = 0.05 #Preguntar por %

In [7]:
def index_of(val, df):
    #max_shift = max_row.applymap(lambda x: df.loc[df[max_row.loc[max_row == x].index[0]] == x].index.get_level_values('Shift').item())
    return df.loc[df[df.columns.get_loc(val)] == val].index.get_level_values(col_shift).item()

def foo(df):
    return None

In [8]:
""" # Normalize each indicator for each country
for country in country_list:
    country_df = df.loc[df.index.get_level_values(col_country) == country]
    for col_name in country_df:
        col = country_df.loc[:, col_name]
        col = col.map(lambda x: (x - col.min()) / (col.max() - col.min()))
        df.loc[df.index.get_level_values(col_country) == country, col_name] = col """

' # Normalize each indicator for each country\nfor country in country_list:\n    country_df = df.loc[df.index.get_level_values(col_country) == country]\n    for col_name in country_df:\n        col = country_df.loc[:, col_name]\n        col = col.map(lambda x: (x - col.min()) / (col.max() - col.min()))\n        df.loc[df.index.get_level_values(col_country) == country, col_name] = col '

In [11]:





for region in region_list:
    region_df = df.loc[df.index.get_level_values(col_region) == region]
    display(region_df)
    countries_of_region = list(set(region_df.index.get_level_values(col_country)))
    countries_of_region.sort()
    for country in countries_of_region:
        country_df = region_df.loc[region_df.index.get_level_values(col_country) == country]
        for col_name in country_df:
            col = country_df.loc[:, col_name]
            col = col.map(lambda x: (x - col.min()) / (col.max() - col.min()))
            region_df.loc[region_df.index.get_level_values(col_country) == country, col_name] = col
    display(region_df.sort_index(level = (col_year, col_country)))
    break

AgriShareGDP  CreditToAgriFishForest  \
Country   Region                Year                                         
Australia East Asia and Pacific 2000       3.94801                2.511352   
                                2001       3.94801                2.263438   
                                2002       2.88853                3.199001   
                                2003       3.06212                3.232433   
                                2004       2.88333                3.493468   
...                                            ...                     ...   
Vanuatu   East Asia and Pacific 2016      20.92800                0.040188   
                                2017      21.37040                0.040098   
                                2018      21.21900                0.072546   
                                2019      21.17310                0.076260   
                                2020      23.55080                0.076260   

                                      EmploymentRural            GDP  \
Country   Region                Year                                   
Australia East Asia and Pacific 2000              NaN  408774.988018   
                                2001              NaN  390108.834372   
                                2002              NaN  435171.313118   
                                2003              NaN  558422.134392   
                                2004              NaN  678337.418023   
...                                               ...            ...   
Vanuatu   East Asia and Pacific 2016              NaN     804.328186   
                                2017              NaN     880.046992   
                                2018              NaN     914.701139   
                                2019              NaN     934.239701   
                                2020              NaN     854.793640   

                                      % Soldiers  Employment in industry  \
Country   Region                Year                                       
Australia East Asia and Pacific 2000    0.537349               21.620001   
                                2001    0.528086               20.740000   
                                2002    0.511698               20.920000   
                                2003    0.529294               20.879999   
                                2004    0.507430               21.240000   
...                                          ...                     ...   
Vanuatu   East Asia and Pacific 2016         NaN               11.890000   
                                2017         NaN               12.670000   
                                2018         NaN               13.390000   
                                2019         NaN               14.080000   
                                2020         NaN               14.080000   

                                      Employment in services  Birth Rate  \
Country   Region                Year                                       
Australia East Asia and Pacific 2000               73.529999      13.000   
                                2001               74.510002      12.700   
                                2002               74.699997      12.800   
                                2003               75.199997      12.600   
                                2004               75.019997      12.300   
...                                                      ...         ...   
Vanuatu   East Asia and Pacific 2016               28.700001      30.424   
                                2017               28.770000      30.015   
                                2018               28.969999      29.595   
                                2019               29.139999      29.175   
                                2020               29.139999      28.763   

                                      Cost business start-up  Death Rate  ...  \
Country   Region           

,,,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,GDP,% Soldiers,Employment in industry,Employment in services,Birth Rate,Cost business start-up,Death Rate,...,Researchers in R&D,R&D expenditure %GDP,% Rural Population,Tertiary School Gender Parity,% Vulnerable female employment,% Vulnerable male employment,Civil Liberties,Freedom of Expression,% Healthcare Investment,Population
Country,Region,Year,,,,,,,,,,,,,,,,,,,,,
Australia,East Asia and Pacific,2000,1.000000,0.115948,NaN,0.015547,1.000000,1.000000,0.000000,0.576923,0.928571,1.000000,...,NaN,NaN,0.936945,NaN,1.000000,0.788530,1.000000,1.000000,0.000000,0.0
Cambodia,East Asia and Pacific,2000,0.982897,0.064862,NaN,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,NaN,NaN,1.000000,NaN,1.000000,1.000000,0.895062,0.986254,0.687351,0.0
China,East Asia and Pacific,2000,NaN,NaN,NaN,0.000000,1.000000,0.123596,0.000000,0.870192,NaN,0.067568,...,0.0,0.000000,1.000000,0.0,1.000000,1.000000,0.307692,0.892473,0.123762,0.0
Fiji,East Asia and Pacific,2000,0.657539,NaN,NaN,0.006543,0.770872,1.000000,0.000000,1.000000,1.000000,0.000000,...,NaN,NaN,1.000000,NaN,0.045580,0.130435,0.863636,0.925156,1.000000,0.0
French Polynesia,East Asia and Pacific,2000,1.000000,NaN,NaN,0.018747,NaN,0.462500,0.000000,1.000000,NaN,0.019517,...,NaN,NaN,0.973879,NaN,1.000000,1.000000,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Singapore,East Asia and Pacific,2020,0.074057,0.356360,NaN,0.874267,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,1.0,0.098016,NaN,NaN,0.352273,0.565218,0.200000,0.666667,0.924474,1.0
Solomon Islands,East Asia and Pacific,2020,0.012130,NaN,NaN,0.976445,NaN,1.000000,1.000000,0.000000,0.000000,0.000000,...,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.882022,1.000000,0.112047,1.0
Thailand,East Asia and Pacific,2020,0.151959,0.016580,NaN,0.899831,0.827266,0.826804,1.000000,0.000000,0.000000,1.000000,...,NaN,1.000000,0.000000,1.0,0.000000,0.017751,0.000000,0.321016,0.718000,1.0


In [10]:
max_list = []
shift_list = []

faulty_list = []

for country in country_list:
    #country = 'Spain'
    #try: 
    if True:
        #print(country)
        corr_list = []
        pval_list = []
        
        
        #Get the DataFrame for a given country.
        country_df = df.loc[df.index.get_level_values(col_country) == country]
        min_year = country_df.index.get_level_values(col_year).min()
        max_year = country_df.index.get_level_values(col_year).max()

        gdp_series = country_df.loc[:, col_gdp]
        ind_df = country_df.drop(col_gdp, axis = 1).dropna(axis = 1)

        #corr_df = ind_df.corrwith(gdp_series, method = 'spearman')
        #display(corr_df)

        for shift in shifts:
            shifted_min = min_year + shift
            shifted_max = max_year - shift

            shifted_gdp = gdp_series.loc[gdp_series.index.get_level_values(col_year) >= shifted_min].reset_index(drop = True)
            shifted_ind = ind_df.loc[ind_df.index.get_level_values(col_year) <= shifted_max].reset_index(drop = True)

            country_corr = shifted_ind.corrwith(shifted_gdp, method = 'spearman')
            pval = shifted_ind.corrwith(shifted_gdp, method = lambda x, y: spearmanr(x, y)[1])
            p = pval.map(lambda x: 1 if x < PVALUE_VAR and x != 0 else np.NaN)
            country_corr = country_corr * p

            country_corr = country_corr.map(lambda x: np.NaN if x == 1.0 or x == -1.0 else x)
            
            display(shifted_ind)
            display(shifted_gdp)
            display(country_corr)
            #display(country_corr_b)
            #display(pval)

            country_corr.name = shift
            corr_list.append(country_corr)
            pval_list.append(pval)

        corr_df = pd.concat(corr_list, axis = 1).transpose().sort_index()
        pval_df = pd.concat(pval_list, axis = 1).transpose().sort_index()

        corr_df.index = pd.MultiIndex.from_tuples((country, sh) for sh, _ in corr_df.iterrows())
        corr_df.index.names = [col_country, 'Shift']

        val = corr_df.apply(lambda x: max(x.min(), x.max(), key=abs))
        val.name = country

        max_row = pd.DataFrame(val).dropna().transpose()
        #display(max_row)

        #ddd = val.map(lambda x: corr_df.loc[corr_df[pd.Series(val).loc[pd.Series(val) == x].index[0]] == x].index.get_level_values('Shift').item())
        #display(corr_df)
        #max_shift = pd.Series(val).map(lambda x: corr_df.loc[corr_df[pd.Series(val).loc[pd.Series(val) == x].index[0]] == x].index.get_level_values('Shift').item())
        #max_shift = max_row.applymap(lambda x: corr_df.loc[corr_df[max_row.loc[max_row == x].index[0]] == x].index.get_level_values('Shift').item())
        
        #print(max_row.index)
        #display(corr_df)
        element = corr_df.loc[:, 'Population'].max()
        #print(element)
        #display(corr_df.loc[corr_df['Population'] == element].index.get_level_values(col_shift).item())
        max_shift = max_row.applymap(lambda x: index_of(x, corr_df))
        

        #print(max_row.loc[max_row == 0.467669])

        #display(max_shift)


        #display(corr_df)
        #display(pval_df)

        max_list.append(max_row)
        shift_list.append(max_shift)
        
    #except Exception: 
    #    faulty_list.append(country)
        
if len(max_list) > 0 and len(shift_list) > 0:
    max_df = pd.concat(max_list, axis = 1).transpose().sort_index()
    shift_df = pd.concat(shift_list, axis = 1).transpose().sort_index()
    #display(max_df)
    #display(shift_df)

#print(faulty_list)



,AgriShareGDP,% Soldiers,Employment in industry,Employment in services,Birth Rate,Cost business start-up,Death Rate,Employment in agriculture,% Female Employment,% Male Employment,...,Maternal Death Risk,Infant Mortality,% Population Growth,% Rural Population,% Vulnerable female employment,% Vulnerable male employment,Civil Liberties,Freedom of Expression,% Healthcare Investment,Population
0,54.0630,7.887961,9.480000,24.680000,48.021,72.0,11.718,65.839996,12.746000,68.787003,...,11.215638,90.5,2.975057,77.922,98.720002,91.879999,0.4000,0.625,1.21,20779957.0
1,54.0630,5.020511,8.980000,24.719999,47.505,72.0,11.387,66.290001,12.868000,68.737000,...,10.604968,87.9,3.902805,77.831,98.760003,92.399998,0.4000,0.625,1.21,21606992.0
2,45.1344,2.153062,9.990000,25.590000,46.901,72.0,11.048,64.419998,12.944000,68.318001,...,9.736509,85.3,4.496719,77.739,98.669999,91.460001,0.4000,0.625,1.21,22600774.0
3,41.9034,2.208290,10.350000,25.950001,46.231,72.0,10.704,63.700001,13.122000,68.320999,...,8.978366,82.7,4.668344,77.647,98.599998,91.039999,0.4030,0.687,5.46,23680871.0
4,35.6128,0.435599,10.610000,26.120001,45.507,72.0,10.356,63.270000,13.301000,68.384003,...,8.315494,80.0,4.321560,77.500,98.549998,90.960003,0.4030,0.677,3.60,24726689.0
5,35.1476,0.416444,11.330000,26.530001,44.723,75.2,10.003,62.150002,13.407000,68.205002,...,7.810411,77.3,3.682700,77.297,98.480003,90.509996,0.4030,0.677,3.37,25654274.0
6,34.3934,0.767568,11.790000,26.930000,43.870,67.4,9.645,61.279999,13.263000,68.384003,...,7.581440,74.6,2.990524,77.093,98.410000,90.249998,0.3970,0.707,2.72,26433058.0
7,35.2660,0.753283,12.200000,27.660000,42.944,84.6,9.287,60.139999,13.073000,68.361000,...,7.268973,71.9,2.493830,76.887,98.309999,89.579998,0.3970,0.708,2.61,27100542.0
8,30.9046,1.364728,12.480000,28.230000,41.949,59.5,8.932,59.290001,12.989000,68.646004,...,6.827597,69.2,2.268273,76.680,98.240000,89.259998,0.4070,0.735,3.00,27722281.0
9,32.7856,3.638823,13.630000,29.480000,40.903,30.2,8.584,56.889999,12.899000,68.572998,...,6.443581,66.7,2.396978,76.472,97.870000,87.760000,0.4040,0.740,2.51,28394806.0


0      3342.034168
1      3598.470576
2      4141.523943
3      4729.042179
4      5388.482107
5      6220.574147
6      7104.711445
7      9412.161765
8     10236.901594
9     11595.133469
10    14698.889678
11    17350.694945
12    19136.499341
13    19621.802455
14    19550.702567
15    18713.048410
16    18019.558182
17    18896.352021
18    18418.848300
19    18904.490263
20    19793.262413
Name: GDP, dtype: float64

AgriShareGDP                     -0.864566
% Soldiers                             NaN
Employment in industry            0.910685
Employment in services            0.911984
Birth Rate                       -0.914286
Cost business start-up           -0.736206
Death Rate                       -0.914286
Employment in agriculture        -0.910685
% Female Employment               0.732468
% Male Employment                -0.705653
Life Expectancy                   0.914286
Maternal Death Risk              -0.905542
Infant Mortality                 -0.914286
% Population Growth                    NaN
% Rural Population               -0.914286
% Vulnerable female employment   -0.910685
% Vulnerable male employment     -0.910685
Civil Liberties                        NaN
Freedom of Expression             0.666451
% Healthcare Investment                NaN
Population                        0.914286
dtype: float64

,AgriShareGDP,% Soldiers,Employment in industry,Employment in services,Birth Rate,Cost business start-up,Death Rate,Employment in agriculture,% Female Employment,% Male Employment,...,Maternal Death Risk,Infant Mortality,% Population Growth,% Rural Population,% Vulnerable female employment,% Vulnerable male employment,Civil Liberties,Freedom of Expression,% Healthcare Investment,Population
0,54.0630,7.887961,9.480000,24.680000,48.021,72.0,11.718,65.839996,12.746000,68.787003,...,11.215638,90.5,2.975057,77.922,98.720002,91.879999,0.4000,0.625,1.21,20779957.0
1,54.0630,5.020511,8.980000,24.719999,47.505,72.0,11.387,66.290001,12.868000,68.737000,...,10.604968,87.9,3.902805,77.831,98.760003,92.399998,0.4000,0.625,1.21,21606992.0
2,45.1344,2.153062,9.990000,25.590000,46.901,72.0,11.048,64.419998,12.944000,68.318001,...,9.736509,85.3,4.496719,77.739,98.669999,91.460001,0.4000,0.625,1.21,22600774.0
3,41.9034,2.208290,10.350000,25.950001,46.231,72.0,10.704,63.700001,13.122000,68.320999,...,8.978366,82.7,4.668344,77.647,98.599998,91.039999,0.4030,0.687,5.46,23680871.0
4,35.6128,0.435599,10.610000,26.120001,45.507,72.0,10.356,63.270000,13.301000,68.384003,...,8.315494,80.0,4.321560,77.500,98.549998,90.960003,0.4030,0.677,3.60,24726689.0
5,35.1476,0.416444,11.330000,26.530001,44.723,75.2,10.003,62.150002,13.407000,68.205002,...,7.810411,77.3,3.682700,77.297,98.480003,90.509996,0.4030,0.677,3.37,25654274.0
6,34.3934,0.767568,11.790000,26.930000,43.870,67.4,9.645,61.279999,13.263000,68.384003,...,7.581440,74.6,2.990524,77.093,98.410000,90.249998,0.3970,0.707,2.72,26433058.0
7,35.2660,0.753283,12.200000,27.660000,42.944,84.6,9.287,60.139999,13.073000,68.361000,...,7.268973,71.9,2.493830,76.887,98.309999,89.579998,0.3970,0.708,2.61,27100542.0
8,30.9046,1.364728,12.480000,28.230000,41.949,59.5,8.932,59.290001,12.989000,68.646004,...,6.827597,69.2,2.268273,76.680,98.240000,89.259998,0.4070,0.735,3.00,27722281.0
9,32.7856,3.638823,13.630000,29.480000,40.903,30.2,8.584,56.889999,12.899000,68.572998,...,6.443581,66.7,2.396978,76.472,97.870000,87.760000,0.4040,0.740,2.51,28394806.0


0      3598.470576
1      4141.523943
2      4729.042179
3      5388.482107
4      6220.574147
5      7104.711445
6      9412.161765
7     10236.901594
8     11595.133469
9     14698.889678
10    17350.694945
11    19136.499341
12    19621.802455
13    19550.702567
14    18713.048410
15    18019.558182
16    18896.352021
17    18418.848300
18    18904.490263
19    19793.262413
Name: GDP, dtype: float64

...

AgriShareGDP                     NaN
% Soldiers                       NaN
Employment in industry           NaN
Employment in services           NaN
Birth Rate                       NaN
Cost business start-up           NaN
Death Rate                       NaN
Employment in agriculture        NaN
% Female Employment              NaN
% Male Employment                NaN
Life Expectancy                  NaN
Maternal Death Risk              NaN
Infant Mortality                 NaN
% Population Growth              NaN
% Rural Population               NaN
% Vulnerable female employment   NaN
% Vulnerable male employment     NaN
Civil Liberties                  NaN
Freedom of Expression            NaN
% Healthcare Investment          NaN
Population                       NaN
dtype: float64

,AgriShareGDP,% Soldiers,Employment in industry,Employment in services,Birth Rate,Cost business start-up,Death Rate,Employment in agriculture,% Female Employment,% Male Employment,...,Maternal Death Risk,Infant Mortality,% Population Growth,% Rural Population,% Vulnerable female employment,% Vulnerable male employment,Civil Liberties,Freedom of Expression,% Healthcare Investment,Population
0,54.063,7.887961,9.48,24.680000,48.021,72.0,11.718,65.839996,12.746,68.787003,...,11.215638,90.5,2.975057,77.922,98.720002,91.879999,0.4,0.625,1.21,20779957.0
1,54.063,5.020511,8.98,24.719999,47.505,72.0,11.387,66.290001,12.868,68.737000,...,10.604968,87.9,3.902805,77.831,98.760003,92.399998,0.4,0.625,1.21,21606992.0


0    18904.490263
1    19793.262413
Name: GDP, dtype: float64

AgriShareGDP                     NaN
% Soldiers                       NaN
Employment in industry           NaN
Employment in services           NaN
Birth Rate                       NaN
Cost business start-up           NaN
Death Rate                       NaN
Employment in agriculture        NaN
% Female Employment              NaN
% Male Employment                NaN
Life Expectancy                  NaN
Maternal Death Risk              NaN
Infant Mortality                 NaN
% Population Growth              NaN
% Rural Population               NaN
% Vulnerable female employment   NaN
% Vulnerable male employment     NaN
Civil Liberties                  NaN
Freedom of Expression            NaN
% Healthcare Investment          NaN
Population                       NaN
dtype: float64

KeyError: -0.8792779862128545

In [ ]:
df1 = pd.DataFrame({1: [9, 3, 8], 2: [1, 5, 5], 3: [2, 2, 1]})
df1

df2 = pd.DataFrame({'t': ['a', 'a', 'b'], 1: [100, 388, 300], 2: [400, 124, 600], 3: [700, 977, 900]})
df2

df1.corrwith(df2)

display(df2)
dfs = df2.copy().loc[:, 1]
display(dfs)

df1.corrwith(dfs)
df2.groupby(['t']).sum()


,t,1,2,3
0,a,100,400,700
1,a,388,124,977
2,b,300,600,900


0    100
1    388
2    300
Name: 1, dtype: int64

,1,2,3
t,,,
a,488,524,1677
b,300,600,900
